In [1]:
import tensorflow as tf

C:\Users\JanU\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images/255.0
test_images=test_images / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

Epoch 1/5
60000/60000 [==============================] - 65s 1ms/step - loss: 0.4944 - acc: 0.8255
Epoch 2/5
60000/60000 [==============================] - 8s 139us/step - loss: 0.3745 - acc: 0.8646
Epoch 3/5
60000/60000 [==============================] - 9s 158us/step - loss: 0.3385 - acc: 0.8779
Epoch 4/5
60000/60000 [==============================] - 8s 140us/step - loss: 0.3132 - acc: 0.8842
Epoch 5/5
10000/10000 [==============================] - 1s 57us/step


## Improving Computer Vision Accuracy using Convolutions

Step 1 is to gather the data. You'll notice that there's a bit of a change here in that the training data needed to be reshaped. That's because the first convolution expects a single tensor containing everything, so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, and the same for the test images. If you don't do this, you'll get an error when training as the Convolutions do not recognize the shape. 

In [6]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

1.9.0


In [8]:
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0


In [10]:
training_images.shape

(60000, 28, 28, 1)

In [11]:
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

Next is to define your model. Now instead of the input layer at the top, you're going to add a Convolution. The parameters are:

    The number of convolutions you want to generate. Purely arbitrary, but good to start with something in the order of 32
    The size of the Convolution, in this case a 3x3 grid
    The activation function to use -- in this case we'll use relu, which you might recall is the equivalent of returning x when x>0, else returning 0
    In the first layer, the shape of the input data.

You'll follow the Convolution with a MaxPooling layer which is then designed to compress the image, while maintaining the content of the features that were highlighted by the convlution. By specifying (2,2) for the MaxPooling, the effect is to quarter the size of the image. Without going into too much detail here, the idea is that it creates a 2x2 array of pixels, and picks the biggest one, thus turning 4 pixels into 1. It repeats this across the image, and in so doing halves the number of horizontal, and halves the number of vertical pixels, effectively reducing the image by 25%.

You can call model.summary() to see the size and shape of the network, and you'll notice that after every MaxPooling layer, the image size is reduced in this way. 

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3),activation='relu', input_shape=(28,28,1)),  #total 32 filter in thing Layer each is of 3x3
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [14]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total para

In [15]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 187s 3ms/step - loss: 0.9477 - acc: 0.6482 1s - loss: 0.9502 - acc - ETA: 0s - loss: 0.9488 - acc: 
Epoch 2/5
60000/60000 [==============================] - 188s 3ms/step - loss: 0.6036 - acc: 0.7710
Epoch 3/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.5310 - acc: 0.8018 3s - loss: 0.5318 - acc: - ETA: 2s
Epoch 4/5
60000/60000 [==============================] - 182s 3ms/step - loss: 0.4883 - acc: 0.8173 2s - lo
Epoch 5/5
60000/60000 [==============================] - 182s 3ms/step - loss: 0.4593 - acc: 0.8302


In [17]:
test_loss,test_acc = model.evaluate(test_images, test_labels)
print('Loss', test_loss)
print('Accuracy', test_acc)

10000/10000 [==============================] - 9s 922us/step
Loss 10.510347602844238
Accuracy 0.3406
